In [ ]:
import commonutils
import warnings

warnings.simplefilter("ignore")

supersetnames = {"BARRIER_HEIGHTS" : \
                       ["BH76","BHDIV10","BHPERI",\
                        "BHROT27","INV24","PX13","WCPT18"] \
                    ,"INTRAMOLECULAR_INTERACTIONS" : \
                       ["ACONF",'Amino20x4',"BUT14DIOL",\
                        "ICONF","IDISP","MCONF",\
                        "PCONF21","SCONF","UPU23"] , \
                    "SMALL_MOLECULES" :\
                        ["AL2X6","ALK8","ALKBDE10","BH76RC",\
                         "DC13","DIPCS10","FH51","G21EA",\
                         "G21IP","G2RC","HEAVYSB11","NBPRC",\
                         "PA26","RC21","SIE4x4","TAUT15",\
                         "W4-11","YBDE18"], \
                    "INTERMOLECULAR_INTERACTIONS" :\
                       ["ADIM6","AHB21","CARBHB12",\
                        "CHB6","HAL59","HEAVY28","IL16",\
                        "PNICO23","RG18","S22","S66","WATER27"] , \
                    "LARGE_SYSTEMS" :\
                        ["BSR36","C60ISO","CDIE20","DARC",\
                         "ISO34","ISOL24","MB16-43","PArel",\
                            "RSE43"]}  

howmanydifs = 3
methods = {}
entries = {"Nuclear Repulsion  :"  : "Nuclear Repulsion", \
        "One Electron Energy:"     : "One Electron Energy", \
        "Two Electron Energy:"     : "Two Electron Energy", \
        "Potential Energy   :"     : "Potential Energy", \
        "Kinetic Energy     :"     : "Kinetic Energy", \
        "E(X)               :"     : "E(X)", \
        "EX              ="        : "E(X)", \
        "E(C)               :"     : "E(C)", \
        "EC              ="        : "E(C)", \
        "Dispersion correction"    : "Dispersion correction" , \
        "FINAL SINGLE POINT ENERGY": "FINAL SINGLE POINT ENERGY"}

functionals = ["M062X", "PBE", "PBE0", "TPSS", "TPSSh", "wB97XD3", "wB97XV"]
basis_sets = ["MINIX", "SVP", "TZVP", "QZVP"]
for func in functionals:
    for basis in basis_sets:
        methods[func + "-" + basis] = []
        for entry in entries:
            methods[func + "-" + basis].append((entry, entries[entry]))

# read all the data and initialize the data structures
rootdir = "../datasets/LiteDataSet/"   
allvalues_perset, fullsetnames, models_results = \
        commonutils.read_and_init (rootdir, supersetnames, howmanydifs, methods, \
                       DEBUG=True)

In [ ]:
import pprint
# check consistency of the data

dims = {}
keysforfunbasis = {}
for k in allvalues_perset.keys():
    #print ("Checking ", k)
    for i in range(len(allvalues_perset[k])):
        #print ("Set ", i)
        #pprint.pprint(allvalues_perset[k][i])
        for kinset in allvalues_perset[k][i].keys():
            if kinset.find("_energydif") >= 0:
                if not kinset in dims:
                    dims[kinset] = set()
                dims[kinset].add(len(allvalues_perset[k][i][kinset]))

                for chiavi in allvalues_perset[k][i][kinset].keys():
                    funcbasis = chiavi.split("_")[0]
                    foundval = chiavi.replace(funcbasis + "_", "")
                    if not funcbasis in keysforfunbasis:
                        keysforfunbasis[funcbasis] = set()
                    keysforfunbasis[funcbasis].add(foundval)

    #print ("Done with ", k)

maxdim = 0
print ("Dimensions")
for k in dims.keys():
    if len(dims[k]) != 1:
        print ("Inconsistent ", k)
    else:
        if maxdim < list(dims[k])[0]:
            maxdim = list(dims[k])[0]
    
for k in dims.keys():
    if len(dims[k]) == 1:
        if list(dims[k])[0] < maxdim:
            print ("Missed values in ", k, list(dims[k])[0], " MAX is ", maxdim)
            funcbasis = k.split("_")[0]
            print("For ", funcbasis)
            for v in keysforfunbasis[funcbasis]:
                print ("  ", v)


In [3]:
# write all the data to a file
#import json
#with open('allvalues_perset.json', 'w') as fp:
#    json.dump(allvalues_perset, fp)

import pickle
pickle.dump(allvalues_perset, open("./data/allvalues_perset.p", "wb"))
pickle.dump(fullsetnames, open("./data/fullsetnames.p", "wb"))
pickle.dump(methods, open("./data/methods.p", "wb"))
pickle.dump(functionals, open("./data/functionals.p", "wb"))
pickle.dump(basis_sets, open("./data/basis_sets.p", "wb"))
pickle.dump(supersetnames, open("./data/supersetnames.p", "wb"))